In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)


CUDA available: True


In [10]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.activation = activation

        self.fc_layers = nn.ModuleList()
        prev_size = input_size
        for hidden_size in hidden_sizes:
            self.fc_layers.append(nn.Linear(prev_size, hidden_size))
            prev_size = hidden_size

        self.output_layer = nn.Linear(prev_size, output_size)

    def forward(self, x):
        x = x.view(-1, self.input_size)
        for fc_layer in self.fc_layers:
            x = self.activation(fc_layer(x))
        x = self.output_layer(x)
        return x

In [11]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

california = fetch_california_housing()
X = california.data
y = california.target

# Create a dataframe with the input and output arrays
df = pd.DataFrame(X, columns=california.feature_names)
df['Target'] = y
# df = df.drop(columns=df.columns[-2])

# Save the dataframe to a CSV file
# df.to_csv('data.csv', index=False)

# Print the shape of the input and output arrays
print("Input shape:", X.shape)
print("Output shape:", y.shape)
# print(y[0])

Input shape: (20640, 8)
Output shape: (20640,)


In [14]:
from sklearn.model_selection import train_test_split
import numpy as np
import time

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using GPU')
else:
    device = torch.device('cpu')
    print('Using CPU')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data into PyTorch tensors and move them to the GPU
X_train = torch.FloatTensor(X_train).to(device)
X_test = torch.FloatTensor(X_test).to(device)
y_train = torch.FloatTensor(y_train).to(device)
y_test = torch.FloatTensor(y_test).to(device)

# Define the model architecture and move it to the GPU
input_size = X_train.shape[1]
hidden_sizes = [5, 2]
try:
    output_size = y_train.shape[1]
except IndexError:
    output_size = 1
model = MLP(input_size, hidden_sizes, output_size, F.relu).to(device)

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Define the loss function
criterion = nn.MSELoss()

# Number of epochs
n_epochs = 300

# Placeholder for losses
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

# Get the current time before the for loop
start_time = time.time()

for it in range(n_epochs):
    # zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    loss.backward()
    optimizer.step()

    # Save losses
    train_losses[it] = loss.item()

    # Test loss
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    test_losses[it] = test_loss.item()

    if (it + 1) % 5 == 0:
        print(f'Epoch {it+1}/{n_epochs}, Train Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}')

# Get the current time after the for loop
end_time = time.time()

# Calculate the time difference
time_difference = end_time - start_time

# Print the time difference
print("Time difference:", time_difference)


Using GPU
Epoch 5/300, Train Loss: 7.1100, Test Loss: 6.7828
Epoch 10/300, Train Loss: 6.0681, Test Loss: 5.7891
Epoch 15/300, Train Loss: 5.2149, Test Loss: 4.9757
Epoch 20/300, Train Loss: 4.5159, Test Loss: 4.3098
Epoch 25/300, Train Loss: 3.9432, Test Loss: 3.7648
Epoch 30/300, Train Loss: 3.4740, Test Loss: 3.3187
Epoch 35/300, Train Loss: 3.0895, Test Loss: 2.9536
Epoch 40/300, Train Loss: 2.7744, Test Loss: 2.6546
Epoch 45/300, Train Loss: 2.5160, Test Loss: 2.4099
Epoch 50/300, Train Loss: 2.3042, Test Loss: 2.2095
Epoch 55/300, Train Loss: 2.1305, Test Loss: 2.0455
Epoch 60/300, Train Loss: 1.9881, Test Loss: 1.9112
Epoch 65/300, Train Loss: 1.8713, Test Loss: 1.8013
Epoch 70/300, Train Loss: 1.7755, Test Loss: 1.7114
Epoch 75/300, Train Loss: 1.6968, Test Loss: 1.6378
Epoch 80/300, Train Loss: 1.6323, Test Loss: 1.5776
Epoch 85/300, Train Loss: 1.5794, Test Loss: 1.5283
Epoch 90/300, Train Loss: 1.5360, Test Loss: 1.4880
Epoch 95/300, Train Loss: 1.5004, Test Loss: 1.4551
Epo

In [13]:
# Set the batch size
batch_size = 32

# Create a DataLoader for the training set
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Number of epochs
n_epochs = 300

# Placeholder for losses
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

# Get the current time before the for loop
start_time = time.time()

for epoch in range(n_epochs):
    # Set the model to training mode
    model.train()

    # Initialize the total loss for this epoch
    total_loss = 0

    # Iterate over the batches of the training set
    for batch_X, batch_y in train_loader:
        # Move the batch to the GPU
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update the total loss for this epoch
        total_loss += loss.item()

    # Calculate the average loss for this epoch
    avg_loss = total_loss / len(train_loader)

    # Save the average loss
    train_losses[epoch] = avg_loss

    # Set the model to evaluation mode
    model.eval()

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass on the test set
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, y_test)

        # Save the test loss
        test_losses[epoch] = test_loss.item()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch {epoch+1}/{n_epochs}, Train Loss: {avg_loss:.4f}, Test Loss: {test_loss.item():.4f}')

# Get the current time after the for loop
end_time = time.time()

# Calculate the time difference
time_difference = end_time - start_time

# Print the time difference
print("Time difference:", time_difference)


/home/tintando/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 5/300, Train Loss: 1.3369, Test Loss: 1.3106
Epoch 10/300, Train Loss: 1.3369, Test Loss: 1.3107
Epoch 15/300, Train Loss: 1.3369, Test Loss: 1.3107
Epoch 20/300, Train Loss: 1.3368, Test Loss: 1.3106
Epoch 25/300, Train Loss: 1.3369, Test Loss: 1.3107
Epoch 30/300, Train Loss: 1.3368, Test Loss: 1.3106
Epoch 35/300, Train Loss: 1.3368, Test Loss: 1.3106
Epoch 40/300, Train Loss: 1.3369, Test Loss: 1.3108
Epoch 45/300, Train Loss: 1.3369, Test Loss: 1.3106
Epoch 50/300, Train Loss: 1.3369, Test Loss: 1.3108
Epoch 55/300, Train Loss: 1.3369, Test Loss: 1.3106
Epoch 60/300, Train Loss: 1.3369, Test Loss: 1.3107
Epoch 65/300, Train Loss: 1.3369, Test Loss: 1.3108
Epoch 70/300, Train Loss: 1.3369, Test Loss: 1.3107


KeyboardInterrupt: 